# Roo Code 문서 데이터 분석

이 노트북은 Roo Code 문서 데이터를 분석하기 위한 것입니다. 아래 코드는 다운로드된 문서 파일들을 읽어서 JSON 형식으로 변환하는 과정을 수행합니다.

## 데이터 수집 과정
1. `step_00_data_download/downloaded_docs/` 폴더에 있는 모든 파일을 탐색합니다.
2. 각 파일에 대해 다음 정보를 수집합니다:
   - `path`: 파일의 상대 경로
   - `name`: 파일 이름
   - `content`: 파일의 전체 내용
3. 수집된 데이터는 리스트 형태로 저장되며, 각 항목은 JSON 객체입니다.

이 데이터는 이후 Roo Code 문서의 구조와 내용을 분석하는 데 사용됩니다.


In [1]:
import os
import json

# downloaded_docs 폴더 경로
downloaded_docs_path = "../step_00_data_download/downloaded_docs/"

# 모든 경로와 파일 읽기
datas = []
for root, dirs, files in os.walk(downloaded_docs_path):
    for file in files:
        file_path = os.path.join(root, file)
        with open(file_path, "r") as f:
            content = f.read()
            
        #file_path대신 downloaded_docs_path 하위의 경로만 저장
        file_path = file_path.replace(downloaded_docs_path, "")
            
        datas.append({"path": file_path, "name": file, "content": content})

print(datas)

[{'path': 'community.md', 'name': 'community.md', 'content': '# Community Projects\n\nWelcome to the Roo Code community section! Here you\'ll find community projects that extend Roo Code\'s capabilities and a gallery of custom modes shared by other users to enhance your development workflow.\n\n## Memory Bank Project by [@GreatScottyMac](https://github.com/GreatScottyMac)\n\nThe [Roo Code Memory Bank](https://github.com/GreatScottyMac/roo-code-memory-bank) project solves a critical challenge in AI-assisted development: **maintaining context across sessions**. By providing a structured memory system integrated with VS Code, it ensures your AI assistant maintains a deep understanding of your project across sessions.\n\n**Key Features**\n\n- 🧠 **Persistent Context**: Remembers project details across sessions and maintains consistent understanding of your codebase\n- 🔄 **Smart Workflows**: Mode-based operation with automatic context switching and project-specific customization\n- 📊 **Knowl

# 데이터 조사

이 노트북에서는 다운로드된 문서 데이터를 분석합니다.

## 데이터 형식
- 데이터는 JSON 형식으로 저장됩니다.
- 각 문서는 다음 필드를 포함합니다:
  - `path`: 파일 경로
  - `name`: 파일 이름
  - `content`: 파일 내용

## 분석 계획
1. 데이터의 content 길이를 내림차순으로 정렬하여 가장 큰 문서부터 조사합니다.
2. 문서 내용의 길이를 분석하여 데이터의 분포를 파악합니다.
3. 문서 유형별 특성을 파악합니다.


In [2]:
# 데이터 분석 코드 작성
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 불러오기 from datas
# content 길이 계산하여 새 컬럼 추가
df = pd.DataFrame(datas)
df['content_length'] = df['content'].apply(len)

# content 길이 기준으로 내림차순 정렬
sorted_data = df.sort_values(by='content_length', ascending=False)

# 상위 10개 문서 출력
print(sorted_data.head(10))

# 문서 내용의 길이 분석


                                     path                    name  \
0                            community.md            community.md   
26   troubleshooting/shell-integration.md    shell-integration.md   
2                                  faq.md                  faq.md   
36         advanced-usage/custom-modes.md         custom-modes.md   
34         advanced-usage/code-actions.md         code-actions.md   
6              basic-usage/using-tools.md          using-tools.md   
39                  advanced-usage/mcp.md                  mcp.md   
35    advanced-usage/model-temperature.md    model-temperature.md   
37  advanced-usage/custom-instructions.md  custom-instructions.md   
38   advanced-usage/prompt-engineering.md   prompt-engineering.md   

                                              content  content_length  
0   # Community Projects\n\nWelcome to the Roo Cod...           14812  
26  # Terminal Shell Integration\n\n## What is She...           14803  
2   # Frequently Asked Q

# 데이터 저장 및 활용

수집된 문서 데이터를 JSONL 형식으로 저장합니다. JSONL은 각 줄이 하나의 JSON 객체를 나타내는 형식으로,
대용량 데이터를 효율적으로 저장하고 처리하는 데 적합합니다.

이 파일은 다음 단계에서 데이터 분석 및 처리를 위해 사용될 예정입니다.
저장된 JSONL 파일은 pandas를 통해 쉽게 불러와서 분석할 수 있습니다.


In [3]:
# datas를 jsonl 파일로 저장
with open("docs.jsonl", "w") as f:
    for data in datas:
        f.write(json.dumps(data) + "\n")


# LangChain을 활용한 데이터셋 구성 및 메타데이터 형성

이번 단계에서는 LangChain의 Document 클래스와 텍스트 분할기(splitter)를 활용하여 
데이터셋을 구성하고 메타데이터를 형성합니다.

## 주요 작업

1. **LangChain Document 객체로 데이터 변환**
   - 원시 데이터를 LangChain의 Document 형식으로 변환
   - 문서 관리 및 처리를 위한 기본 구조 생성

2. **적절한 텍스트 분할기 선택 및 적용**
   - 문서 특성에 맞는 분할 전략 선택
   - 효율적인 검색을 위한 최적 청크 크기 설정

3. **문서별 메타데이터 추가 및 구조화**
   - 출처, 카테고리, 생성일자 등 메타정보 추가
   - 검색 및 필터링을 위한 메타데이터 구조화


In [4]:
import json
import pandas as pd
from langchain.schema import Document
from langchain_text_splitters import MarkdownHeaderTextSplitter

# JSONL 파일 로드
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# 데이터를 LangChain Document 객체로 변환
def convert_to_documents(data):
    documents = []
    for item in data:
        doc = Document(
            page_content=item.get('content', ''),
            metadata={
                'path': item.get('path', ''),
                'name': item.get('name', '')
            }
        )
        documents.append(doc)
    return documents

# 마크다운 헤더 기준으로 문서 분할 함수
def split_by_markdown_headers(documents):
    # 헤더 분할 설정 (# 및 ## 기준)
    headers_to_split_on = [
        ("#", "header1"),
        ("##", "header2")
    ]
    
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
    
    split_docs = []
    for doc in documents:
        try:
            doc_splits = markdown_splitter.split_text(doc.page_content)
            for split in doc_splits:
                # 원본 메타데이터 유지하면서 헤더 정보 추가
                metadata = doc.metadata.copy()
                metadata.update(split.metadata)
                
                split_doc = Document(
                    page_content=split.page_content,
                    metadata=metadata
                )
                split_docs.append(split_doc)
        except Exception as e:
            print(f"문서 분할 중 오류 발생: {e}")
            # 오류 발생 시 원본 문서 그대로 추가
            split_docs.append(doc)
    
    return split_docs

# JSONL 파일 로드
data = load_jsonl("docs.jsonl")
print(f"로드된 문서 수: {len(data)}")

# LangChain Document 객체로 변환
documents = convert_to_documents(data)
print(f"변환된 Document 객체 수: {len(documents)}")

# 마크다운 헤더 기준으로 문서 분할
split_docs = split_by_markdown_headers(documents)
print(f"헤더 기준 분할로 생성된 청크 수: {len(split_docs)}")

# 분할된 문서의 예시 확인
if len(split_docs) > 0:
    print("\n헤더 분할 첫 번째 청크 예시:")
    print(f"내용: {split_docs[0].page_content[:150]}...")
    print(f"메타데이터: {split_docs[0].metadata}")

# 분할된 문서를 데이터프레임으로 변환하여 확인
split_docs_df = pd.DataFrame([
    {
        'content': doc.page_content,
        'path': doc.metadata.get('path', ''),
        'name': doc.metadata.get('name', ''),
        'length': len(doc.page_content)
    } for doc in split_docs
])

# 데이터프레임 기본 정보 출력
print("\n분할된 문서 데이터프레임 정보:")
print(split_docs_df.info())
print("\n분할된 문서 길이 통계:")
print(split_docs_df['length'].describe())

# 분할된 문서를 JSONL 파일로 저장
with open("split_docs.jsonl", "w") as f:
    for doc in split_docs:
        doc_dict = {
            'content': doc.page_content,
            'metadata': doc.metadata
        }
        f.write(json.dumps(doc_dict) + "\n")

print("\n분할된 문서가 'split_docs.jsonl' 파일로 저장되었습니다.")

로드된 문서 수: 44
변환된 Document 객체 수: 44
헤더 기준 분할로 생성된 청크 수: 237

헤더 분할 첫 번째 청크 예시:
내용: # Community Projects  
Welcome to the Roo Code community section! Here you'll find community projects that extend Roo Code's capabilities and a galler...
메타데이터: {'path': 'community.md', 'name': 'community.md', 'header1': 'Community Projects'}

분할된 문서 데이터프레임 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  237 non-null    object
 1   path     237 non-null    object
 2   name     237 non-null    object
 3   length   237 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 7.5+ KB
None

분할된 문서 길이 통계:
count      237.000000
mean       630.827004
std        933.871906
min         28.000000
25%        260.000000
50%        471.000000
75%        701.000000
max      12356.000000
Name: length, dtype: float64

분할된 문서가 'split_docs.jsonl' 파일로 저장되었습니다.
